In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6174,2024-07-21,Eua Nba,00:00,Dallas Mavericks,Oklahoma City Thunder,1.69,2.15,176.5,1.87,1.91,1.5,1.81,1.99,04ESROsm,0.591716,0.465116,0.534759,0.523560,0.056832,0.4,0.2,NaN,NaN,241.366,62.913613,0.260656,198.312,60.617797,0.305669,192.976,204.188,235.46,144.90,0.0,0.0,0.0,0.0,0.169411,0.014965,0.066989,0.18,0.036,19.166667,0.598514,0.8,0.201486,-1.45,-0.290,-3.965517,0.490631,0.5,0.009369
6175,2024-07-21,Europa Campeonato Europeu Sub,09:30,Hungria U20,Geórgia U20,1.86,1.86,155.5,1.82,1.88,-1.5,1.99,1.97,jRndPq1E,0.537634,0.537634,0.549451,0.531915,0.075269,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.022933,0.007142,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6176,2024-07-21,Mundo Amistoso Internacional,16:00,França,Austrália,1.64,2.18,158.5,1.86,1.84,-2.5,1.83,2.37,ShuSCjPb,0.609756,0.458716,0.537634,0.543478,0.068472,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,144.54,0.00,0.0,0.0,0.0,0.0,0.199915,0.007644,0.181827,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6177,2024-07-21,América Campeonato Mundial,20:30,Guatemala,Honduras,2.17,1.66,138.5,1.87,1.87,1.5,1.98,1.52,zosiQLio,0.460829,0.602410,0.534759,0.534759,0.063239,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.188316,0.000000,0.185868,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6178,2024-07-21,Eua Nba,16:00,Minnesota Timberwolves,Orlando Magic,1.77,2.10,180.5,1.91,1.91,-2.5,1.95,2.40,pCQKGwIl,0.564972,0.476190,0.523560,0.523560,0.041162,0.8,0.8,NaN,NaN,179.266,41.892831,0.233691,211.228,43.545111,0.206152,168.148,201.968,135.90,226.54,0.0,0.0,0.0,0.0,0.120592,0.000000,0.146298,-3.22,-0.644,-1.195652,0.605523,0.4,-0.205523,-5.00,-1.000,-1.100000,0.517851,0.3,-0.217851
6179,2024-07-21,Eua Nba,18:00,Boston Celtics,Philadelphia 76ers,2.38,1.63,181.5,1.91,1.91,2.5,2.00,1.44,Uq6qOtsC,0.420168,0.613497,0.523560,0.523560,0.033665,0.8,1.0,NaN,NaN,183.912,46.119425,0.250769,208.708,47.472669,0.227460,173.620,226.436,238.26,207.36,0.0,0.0,0.0,0.0,0.264504,0.000000,0.230221,-0.27,-0.054,-25.555556,0.793537,0.8,0.006463,2.10,0.420,1.500000,0.474155,0.6,0.125845
6180,2024-07-21,Eua Nba,19:30,Brooklyn Nets,Charlotte Hornets,2.18,1.73,175.5,1.91,1.91,1.5,2.00,1.56,0rwhFobs,0.458716,0.578035,0.523560,0.523560,0.036750,0.2,0.6,NaN,NaN,166.704,31.536155,0.189175,206.128,82.685521,0.401137,232.824,403.148,155.48,349.86,0.0,0.0,0.0,0.0,0.162761,0.000000,0.174790,1.86,0.372,3.172043,0.535260,0.6,0.064740,-3.22,-0.644,-1.133540,0.290048,0.2,-0.090048
6181,2024-07-21,Eua Nba,20:00,San Antonio Spurs,Toronto Raptors,1.73,2.18,172.5,1.91,1.91,-3.5,2.00,2.75,WSnMK7zK,0.578035,0.458716,0.523560,0.523560,0.036750,0.6,0.4,NaN,NaN,175.458,34.457239,0.196385,223.736,150.568462,0.672974,249.366,238.732,134.40,129.20,0.0,0.0,0.0,0.0,0.162761,0.000000,0.223297,-1.13,-0.226,-3.230088,0.427705,0.4,-0.027705,-3.60,-0.720,-1.638889,0.388922,0.1,-0.288922
6182,2024-07-21,Eua Nba,21:30,Utah Jazz,Detroit Pistons,2.10,1.77,173.5,1.91,1.91,1.5,2.00,1.57,UNU8B3c6,0.476190,0.564972,0.523560,0.523560,0.041162,0.6,0.2,NaN,NaN,203.570,15.074860,0.074052,180.696,18.894790,0.104567,217.972,488.300,183.18,195.84,0.0,0.0,0.0,0.0,0.120592,0.000000,0.170339,-1.09,-0.218,-5.045872,0.522736,0.4,-0.122736,0.30,0.060,12.833333,0.238979,0.4,0.161021
6183,2024-07-21,Eua Nba,22:00,Portland

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,09:30,Europa Campeonato Europeu Sub,Hungria U20,Geórgia U20,155.5,1.82,0.9934,Over
1,16:00,Mundo Amistoso Internacional,França,Austrália,158.5,1.86,1.0000,Over
2,16:00,Eua Nba,Minnesota Timberwolves,Orlando Magic,180.5,1.91,1.0000,Over
3,20:00,Eua Nba,San Antonio Spurs,Toronto Raptors,172.5,1.91,1.0000,Over
4,11:00,Europa Eurobasket Sub,Itália U20,Israel U20,151.5,1.83,0.9983,Over
